In [1]:
import weapon_data as weapons

D:\z_outsourced_programs\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_data, test_data = weapons.get_data(2, #num_of_categorical_param
                                         15, #num_of_numerical_param
                                         1, #ammo_feature_column_dimension
                                         debug=True) 
print(train_data.standardized_max_values)
print(train_data.standardized_min_values)
print(test_data.standardized_max_values)
print(test_data.standardized_min_values)

Feature columns to tensor variables dictionary:
_NumericColumn(key='damages_first', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='damages_last', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='dmg_distances_first', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='dmg_distances_last', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0)) = []
_NumericColumn(key='hiprecoildec', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='hiprecoilright', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) = []
_NumericColumn(key='hiprecoilup', shape=(1,), default_value=None,

In [3]:
print("Training data size %i "%train_data.num_examples)
print("Test data size %i "%test_data.num_examples)
print("Sum of data %i"%(train_data.num_examples+test_data.num_examples))

Training data size 127 
Test data size 21 
Sum of data 148


In [ ]:
import tensorflow as tf

def addGeneratedWeapon(generatedWeaponTensorDict, dictToAdd):
    for key, value in generatedWeaponTensorDict.items():
        if key not in dictToAdd:
            dictToAdd[key] = []
        dictToAdd[key].append(value)

def check_data(data):
    genDict = {}

    #print to see if normalized
    print(data.data[0])
    
    #check if unnormalizing and adding works
    var_init = tf.global_variables_initializer()

    batch_size = 2
    with tf.Session() as sess:
        sess.run(var_init)

        for i in range(0,1):
            batch = data.next_batch(batch_size, shuffle=True)
            #print("Batch size %i" %len(batch))
            data.add_new_weapon(batch[0])

            
        tensorDict = [data.decode_processed_tensor(b)[0] for b in batch]
        [addGeneratedWeapon(t, genDict) for t in tensorDict]
        #debug_printDict(tensorDict[0])


    weapons.debug_printDict(genDict)


In [ ]:
check_data(train_data)
check_data(test_data)